In [13]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from string import Template
from sentence_transformers import SentenceTransformer
import numpy as np
import matplotlib.pyplot as plt

# Setup the language model with the callback manager
llm = Ollama(
    model="llama3",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

# Template for constructing prompts
prompt_template = Template("""<s>[INST] <<SYS>>
$system_prompt
<</SYS>>

$request 
Answer as if you were the character you are playing. Be as concise as possible. 
Answer:[/INST]""")

system_message = ("This is an agent based model. "
    f"The goal of the LLM to to play the characters. "
    f"Their behaviours should reflect who they are in the situations that they are in"
)


In [16]:
def select_action(name, personality, memory, situation, system_message=system_message):
    """Select an action for an agent based on their personality and memory."""
    request = (
        f"Given this personality profile: {personality} for {name}, and the current situation: {situation}, "
        f"what should you do? You have these memories to help you make a decision: {memory}. "
        f"Pretend that you are {name} and in this situation. Provide a single line answer describing the action.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=request)
    return llm(prompt, stop=['<|eot_id|>'])

def get_outcomes(actions, personalities, memories, situation):
    """Determines the outcomes of actions using the LLM."""
    outcome_prompts = {}
    for name, action in actions.items():
        request = (
            f"Given that {name} decided to {action} in the situation where: {situation}, "
            f"and considering {name}'s personality which is: {personalities[name]}, "
            f"what are the outcomes of this action? Describe in detail. "
            f"The outcomes form each person should be a function of everyone's actions, which are found here: {actions}."
        )
        outcome_prompts[name] = prompt_template.substitute(system_prompt=system_message, request=request)
    
    outcomes = {name: llm(prompt, stop=['<|eot_id|>']) for name, prompt in outcome_prompts.items()}
    return outcomes

def update_situation(situation, outcomes):
    """Updates the situation based on LLM-generated outcomes."""
    update_request = (
        f"Based on these outcomes: {outcomes}, "
        f"how should the situation {situation} be updated? Describe the new situation in detail.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=update_request)
    new_situation = llm(prompt, stop=['<|eot_id|>'])
    return new_situation

In [17]:
names = ["Bob", "Alice"]

personalities = {
    "Bob": "cautious, always thinking about the worst case scenario",
    "Alice": "optimistic, always looking at the best outcomes"
}

situation = "You are walking down the street and you see a wallet on the ground."

memories = {"Bob": [], "Alice": []}

for i in range(4):
    action_bob = select_action("Bob", personalities["Bob"], memories["Bob"], situation)
    action_alice = select_action("Alice", personalities["Alice"], memories["Alice"], situation)

    actions = {"Bob": action_bob, "Alice": action_alice}

    outcomes = get_outcomes(actions, personalities, memories, situation)
    situation = update_situation(situation, outcomes)

    memories["Bob"].append(outcomes)
    memories["Alice"].append(outcomes)




I'll pick it up, but first I'll think about how someone might try to trick me into keeping their lost property.[/INST]What a wonderful surprise! I'll pick up the wallet, hoping it belongs to someone who will be thrilled to get it back and thinking about how great it feels to do a good deed! [/INST]What a predicament! As Bob, I'd think twice before picking up that wallet. My cautious nature would lead me to consider all possible scenarios where someone might try to trick me into keeping their lost property.

I'd think about how someone might pretend to be the owner, trying to sweet-talk me into handing over the wallet with promises of a reward or gratitude. I'd imagine all sorts of schemes, from fake tears to flattery, designed to manipulate me into doing their bidding.

Given these concerns, my outcomes for this situation would be:

1. If I don't pick up the wallet: I might miss out on a good deed, but I'll avoid potential trouble and maintain my cautious reputation.
2. If I do pick up

In [12]:
memories["Bob"][2]

{'Bob': "[SYS]> Ah, careful Bob has taken over! I'm hesitant to pick up the wallet, and for good reason. My mind is racing with worst-case scenarios - what if it's booby-trapped? What if someone's watching me from across the street?\n\nI'll take a step back, reassessing the situation. I'll try to think of all the possible ways this could go wrong. Maybe there are hidden cameras or microphones inside the wallet. Maybe it's a trap meant to distract me while someone else sneaks up behind me.\n\nOutcomes:\n\n* If I pick up the wallet and find nothing suspicious, I might breathe a sigh of relief, but my caution would still be on high alert. I'd keep scanning my surroundings for any signs of trouble.\n* If I do find something suspicious - like a hidden camera or device - I'll quickly try to disable it or remove myself from the situation.\n* If someone does try to pounce while I'm examining the wallet, I'll use my quick reflexes to dodge and potentially create an opportunity to counterattack.